# Stroke type proportions for patients with onset-to-arrival times below various cutoffs

Find the proportions of patients with nLVO, LVO, and haemorrhagic stroke in subgroups of patients that arrive at hospital by ambulance within X hours of their stroke onset.

## Plain English summary

Generally we expect people who have more severe strokes to recognise the symptoms and arrive at hospital sooner than people who have mild strokes. In the modelling, we usually use the stroke severity score to label patients as having an nLVO (for milder strokes) or an LVO (for more severe strokes). This means that a subgroup of patients who arrive at hospital very quickly, within a few hours after their stroke onset, is likely to contain more LVO patients than a subgroup of patients who arrived more slowly, within 24 hours.

In this notebook we check the proportions of stroke types in subgroups of patients arriving by various cutoff times. The times are one hour after onset, two hours after onset, and so on up to and including 24 hours after onset.

We expect to see some difference in the proportions across subgroups and so these calculations are more of a check to see how big the differences are.

## Aims

Calculate the proportions of patients with nLVO, LVO, and haemorrhagic stroke in subgroups that arrive by various cutoff times. Look at subgroups arriving before 1 hour, 2 hour, ..., 24 hours.

Save the resulting proportions to file and make pie charts to compare the variations visually.

## Method

Load existing data on proportions of stroke types arriving by the expected time. Simplify the data into just three values for each time: proportion of those who arrived on time with nLVO, LVO, and haemorrhagic strokes. Compare the results across the cutoff times using pie charts.

## Code setup

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Load data

In [2]:
df_props_full = pd.read_csv('stroke_type_proportions_with_arrival_time.csv', index_col=0)

In [3]:
df_props_full.head(3).T

expected_time,1.0,2.0,3.0
prop_arriving_within_expected_time,0.05991,0.26796,0.37680
prop_haemo_of_prop_arriving_within_expected_time,0.15957,0.15649,0.15302
prop_nlvo_of_prop_arriving_within_expected_time,0.49172,0.54775,0.57470
prop_lvo_of_prop_arriving_within_expected_time,0.34871,0.29576,0.27227
prop_haemo_arriving_within_expected_time,0.00956,0.04193,0.05766
prop_nlvo_arriving_within_expected_time,0.02946,0.14678,0.21655
prop_lvo_arriving_within_expected_time,0.02089,0.07925,0.10259
prop_haemo_of_prop_arriving_after_expected_time,0.13094,0.12393,0.12034
prop_nlvo_of_prop_arriving_after_expected_time,0.66410,0.69258,0.70157
prop_lvo_of_prop_arriving_after_expected_time,0.20496,0.18349,0.17809


## Simplify data

Keep only patients who arrived before the onset-to-arrival time cutoff, and store proportions of nLVO, LVO, and haemorrhagic stroke for each time to sum to 1.

In [4]:
df_props = df_props_full.copy()

df_props['prop_nlvo'] = (
    df_props['prop_nlvo_arriving_within_expected_time'] /
    df_props['prop_arriving_within_expected_time']
)

df_props['prop_lvo'] = (
    df_props['prop_lvo_arriving_within_expected_time'] /
    df_props['prop_arriving_within_expected_time']
)

df_props['prop_haemo'] = (
    df_props['prop_haemo_arriving_within_expected_time'] /
    df_props['prop_arriving_within_expected_time']
)

df_props = df_props[['prop_nlvo', 'prop_lvo', 'prop_haemo']]

In [5]:
df_props

,prop_nlvo,prop_lvo,prop_haemo
expected_time,,,
1.0,0.491738,0.348690,0.159573
2.0,0.547768,0.295753,0.156479
3.0,0.574708,0.272266,0.153025
4.0,0.588490,0.261193,0.150318
5.0,0.597228,0.254803,0.147948
6.0,0.603558,0.250317,0.146125
7.0,0.608758,0.246580,0.144662
8.0,0.612585,0.243890,0.143506
9.0,0.616013,0.241457,0.142529


# Extra subgroup: patients who arrive eventually

Have no limits on the arrival time, and so include patients who are missing the onset-to-arrival time data.

The total number of patients and the patient proportions will be the same across all time cutoffs because we're taking everybody whether they're before or after the cutoff. (Except for any rounding errors!)

In [6]:
df_props_extra = df_props_full.copy()

df_props_extra['prop_haemo'] = (
    df_props_extra['prop_haemo_arriving_within_expected_time'] +
    df_props_extra['prop_haemo_arriving_after_expected_time']
)

df_props_extra['prop_nlvo'] = (
    df_props_extra['prop_nlvo_arriving_within_expected_time'] +
    df_props_extra['prop_nlvo_arriving_after_expected_time']
)

df_props_extra['prop_lvo'] = (
    df_props_extra['prop_lvo_arriving_within_expected_time'] +
    df_props_extra['prop_lvo_arriving_after_expected_time']
)

df_props_extra['prop_sum'] = (
    df_props_extra['prop_lvo'] +
    df_props_extra['prop_nlvo'] + 
    df_props_extra['prop_haemo']
)

# Only keep these columns:
df_props_extra = df_props_extra[['prop_haemo', 'prop_nlvo', 'prop_lvo', 'prop_sum']]

In [7]:
df_props_extra

,prop_haemo,prop_nlvo,prop_lvo,prop_sum
expected_time,,,,
1.0,0.13266,0.65377,0.21358,1.00001
2.0,0.13265,0.65377,0.21357,0.99999
3.0,0.13266,0.65377,0.21357,1.00000
4.0,0.13266,0.65377,0.21358,1.00001
5.0,0.13266,0.65376,0.21357,0.99999
6.0,0.13265,0.65377,0.21358,1.00000
7.0,0.13265,0.65377,0.21357,0.99999
8.0,0.13265,0.65377,0.21357,0.99999
9.0,0.13266,0.65376,0.21358,1.00000


Pick the first option that has a sum of 1 exactly:

In [8]:
ind = df_props_extra[df_props_extra['prop_sum'] == 1.0].index.values[0]

Place this row into the full results list:

In [9]:
df_props.loc['inf'] = df_props_extra.loc[ind, ['prop_nlvo', 'prop_lvo', 'prop_haemo']]

In [10]:
df_props.tail(3)

,prop_nlvo,prop_lvo,prop_haemo
expected_time,,,
23.0,0.635572,0.228598,0.135846
24.0,0.636603,0.227869,0.135543
inf,0.653770,0.213570,0.132660


## Save results

Round all values to 5 decimal places, which should be plenty:

In [11]:
df_props = np.round(df_props, 5)

In [12]:
df_props.to_csv('stroke_type_props_arrived_on_time.csv')

## Plot proportions

In [13]:
for i in range(len(df_props)):
    fig, ax = plt.subplots()
    
    time = df_props.index.values[i]
    ax.pie(df_props.loc[time].values, labels=df_props.columns, autopct='%1.1f%%')
    try:
        title = f'Expected onset-to-arrival time less than {time:.0f} hours'
        savename = f'props_{time:.0f}h'
    except ValueError:
        title = f'All patients who arrive eventually'
        savename = 'props_eventually'
    ax.set_title(title)
    
    plt.savefig(f'./pie_frames/{savename}.png', bbox_inches='tight')
    plt.close()

## Show the results for 6 hours and 24 hours:

![Pie chart of stroke type proportions for patients arriving within 6 hours of stroke onset.](./pie_frames/props_6h.png)
![Pie chart of stroke type proportions for patients arriving within 24 hours of stroke onset.](./pie_frames/props_24h.png)

The differences are fairly small, plus or minus only a few percent...

In [14]:
df_props.loc[[6.0, 24.0]]

,prop_nlvo,prop_lvo,prop_haemo
expected_time,,,
6.0,0.60356,0.25032,0.14613
24.0,0.63660,0.22787,0.13554


... so generally speaking it should be fine to pick one set of stroke type proportions at an expected onset-to-arrival time and apply that one set to all results.

## Show the results for all times:

![Animated pie chart of stroke type proportions for patients arriving within the cutoff time stroke onset. The cutoff times run from 1 hour to 24 hours in steps of 1 hour.](./pie_frames/props_with_time.gif)


## Show the results for any arrival time:

![Pie chart of stroke type proportions for patients arriving at any time after stroke onset.](./pie_frames/props_eventually.png)


## Conclusion

There is a small variation of the proportions of different stroke types in subgroups of patients who arrive at hospital within a given time of their stroke onset.

The variation between patients arriving within 6 hours and patients arriving within 24 hours is quite small. The proportions for the 6 hours group could be applied to results for the 24 hours group without a huge loss of accuracy.